In [32]:
import openai
import dotenv
import os
import requests
import yaml
import tqdm
import json
from caseconverter import macrocase

dotenv.load_dotenv()

client = openai.OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)


In [33]:
items_pdata_path = "../.projor/data/items.pdata.yaml"
with open(items_pdata_path, "r") as f:
    items = yaml.safe_load(f)['objects']

In [34]:
def capital_case(macrocasename):
    parts = macrocasename.split(" ")
    capitalized_parts = [part.capitalize() for part in parts]
    return " ".join(capitalized_parts)
def kebab_case(macrocasename):
    parts = macrocasename.split(" ")
    kebab_parts = [part.lower() for part in parts]
    return "-".join(kebab_parts)
def render_prompt(itemname):
    return [
        {
            "role": "system",
            "content": "Your task is to generate the description of an item in a game. Make it a funny, and entertaining name, use metaphors or pop culture references. Use only a single sentence, keep the descriptions short and concise. DO NOT start the description with the name of the element! Respond with nothing else, just a JSON object with the description key."
        },
        {
            "role": "user",
            "content": "Water"
        },
        {
            "role": "assistant",
            "content": """{"description": "It is the liquid substance, that is absolutely necessary for life."}"""
        },
        {
            "role": "user",
            "content": capital_case(itemname)
        }
    ]

In [35]:
def gen_description_with_retry(prompt, outfilename):
    for i in range(10):
        try:
            response = client.chat.completions.create(
                messages=render_prompt(prompt),
                model="gpt-4o",
                max_tokens=500,
                response_format={
                    "type": "json_object",
                }
            )

            response_json_str = response.choices[0].message.content
            response_json = json.loads(response_json_str)
            with open(outfilename, "w") as f:
                f.write(response_json['description'])
        
            return
        except Exception as e:
            print(f"Retrying for prompt '{prompt}' and filename '{outfilename}' due to exception: {e} ...")
            continue
    print(f"Failed to generate description for prompt '{prompt}' and filename '{outfilename}'")
    raise Exception(f"Failed to generate description for prompt '{prompt}' and filename '{outfilename}'")

In [36]:
for item in tqdm.tqdm(items):
    itemname = item['name']
    outfilename = f"../.projor/tmp/{kebab_case(itemname)}.description"
    if os.path.exists(outfilename):
        print(f"Skipping '{itemname}' as description already exists.")
        continue
    gen_description_with_retry(itemname, outfilename)

100%|██████████| 172/172 [03:28<00:00,  1.21s/it]
